# How to Train YOLOv5 on Custom Objects

https://models.roboflow.com/object-detection/yolov5

https://colab.research.google.com/drive/1gDZ2xcTOgR39tGGs-EZ6i3RTs16wmzZQ

# How to Train YOLOv5 on Custom Objects

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference


# 1. Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

- yolov5 환경을 설치한다.

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

- 관련 내용을 설치한다.

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# 2. Download Correctly Formatted Custom Dataset 




![YOLOv5 PyTorch export](https://i.imgur.com/5vr9G2u.png)


- roboflow에서 우리가 필요로 하는 mask dataset을 다운로드 한다.

In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install -q roboflow
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="roboflow-yolov5")
!curl -L "https://public.roboflow.com/ds/q17S9NnLHc?key=UlRVud9ghQ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


- 디렉토리를 yolov5로 이동한다.

In [ ]:
%cd /content/yolov5

- yaml 화일에 들어있는 이미지 데이터의 경로를 확인한다.

In [ ]:
%cat data.yaml

# 3. Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

다음은 YOLOv5 모델 구성 파일을 수정하는 과정이다. custom_yolov5s.yaml.

- yolov5s.yaml의 내용 중에서  parameters의 nc를 수정하고, 별도의 custom_yolov5s.yaml을 만든다

In [6]:
# define number of classes based on YAML
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

- yolov5s.yaml의 내용을 확인한다.

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov5s.yaml

- writetemplate 명령어를 활성화한다.

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

- yolov5s.yaml의 내용을 복사하여 custom_yolov5s.yaml 을 만들고, nc에서 80을 지우고  {num_classes}로 바꾼다.

In [9]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

- custom_yolov5s.yaml의 내용을 출력하여 nc의 값이 2로 바뀌었는지 확인하다.

- 다음은 완성된 YOLOv5 모델 구성 파일이다 (custom_yolov5s.yaml)

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

- data.yaml의 경로를 확인한다.

In [ ]:
%cat data.yaml

# 중요. 무엇이 잘못 되었는지 확인해 보자!

# 4. Train Custom YOLOv5 Detector

- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

data.yaml 및 custom_yolov5s.yaml 이 준비되면 교육을 시작할 수 있다.

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data /content/yolov5/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

# 5. Evaluate Custom YOLOv5 Detector Performance

- tensorboard 로 성능 확인하기

In [ ]:
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

- results.png 그래프

어떤 이유로든 Tensorboard를 시각화할 수 없다면,  결과를 플로팅 utils.plot_results 한다.

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results3/results.png', width=1000)  # view results.png

# 6. Visualize Our Training Data with Labels


원본 이미지와 증강 이미지를 시각화할 수 있다.

- 원본 이미지 출력

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results3/test_batch0_labels.jpg', width=900)

- 증강 이미지 출력

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results3/train_batch0.jpg', width=900)

# 7. Run Inference  With Trained Weights


추론을 시작하기 위해 다음 매개변수를 사용하여 명령을 실행한다.

1. conf: 예측을 위한 모델 신뢰도(높은 신뢰도가 필요한 경우 예측이 적음)
2. source: 이미지 디렉토리, 개별 이미지, 비디오 파일 및 장치의 웹캠 포트를 허용할 수 있다.
3. weights:weights/ 여기에서 폴더 에서 사용할 모델을 지정한다.
4. name: 여기에서 모델의 다른 클래스 이름을 지정한다(YAML 파일을 기반으로 노트북에서 이전에 생성함)

- 훈련된 모델(best.pt 또는 last.pt)로 추론하기

In [ ]:
%ls /content/yolov5/runs/train/yolov5s_results3/weights

- detect 하기

이제 훈련된 모델을 사용하여 Roboflow에서 데이터 세트를 다운로드했을 때의 테스트 이미지에 대한 추론을 수행한다. 훈련 과정의 모든 가중치는 에서 찾을 수 있다   weights/.

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/yolov5/runs/train/yolov5s_results3/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/test/images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# 8. Export Trained Weights for Future Inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [33]:
%cp /content/yolov5/runs/train/yolov5s_results3/weights/best.pt /content/gdrive/My\ Drive